In [1]:
# Cell 1: Cài đặt Thư viện Cần Thiết
!pip install datasets torch numpy seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=5bd0523e58428d6874c7b42122b6157993c665ef4b5cca49ed3aa2fb440b5837
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [5]:
# Cell 2: Tải và Tiền xử lý Dữ liệu (SỬA LỖI ATTRIBUTEERROR)
import torch
from datasets import load_dataset
from collections import Counter

# 1. Tải dữ liệu CoNLL 2003 từ phiên bản mirror đã được lưu trữ (ổn định)
print("Đang tải bộ dữ liệu CoNLL 2003 (phiên bản ổn định)...")
dataset = load_dataset("lhoestq/conll2003")
print("Tải dữ liệu hoàn tất.")
print(dataset)

# 2. Trích xuất câu và nhãn
train_sentences = dataset["train"]["tokens"]
train_tags_numerical = dataset["train"]["ner_tags"]
val_sentences = dataset["validation"]["tokens"]
val_tags_numerical = dataset["validation"]["ner_tags"]

# Sửa lỗi AttributeError: Định nghĩa tên nhãn CoNLL 2003 tiêu chuẩn (9 classes)
tag_names = [
    'O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'
]
print(f"\nCác nhãn NER (9 classes): {tag_names}")

# Chuyển đổi nhãn số sang nhãn string (sử dụng tag_names đã định nghĩa)
def convert_numerical_tags_to_string(numerical_tags_list):
    string_tags_list = []
    for tags in numerical_tags_list:
        string_tags_list.append([tag_names[tag] for tag in tags])
    return string_tags_list

train_tags = convert_numerical_tags_to_string(train_tags_numerical)
val_tags = convert_numerical_tags_to_string(val_tags_numerical)

# 3. Xây dựng Từ điển (Vocabulary)
word_counts = Counter(word for sentence in train_sentences for word in sentence)
unique_words = sorted(word_counts.keys())

# Ánh xạ từ -> index
word_to_ix = {"<PAD>": 0, "<UNK>": 1}
for word in unique_words:
    if word not in word_to_ix:
        word_to_ix[word] = len(word_to_ix)

# Ánh xạ nhãn -> index (dùng tag_names đã định nghĩa)
tag_to_ix = {}
for tag in tag_names:
    if tag not in tag_to_ix:
        tag_to_ix[tag] = len(tag_to_ix)

ix_to_tag = {v: k for k, v in tag_to_ix.items()}

# In ra kích thước từ điển
VOCAB_SIZE = len(word_to_ix)
OUTPUT_SIZE = len(tag_to_ix)
print("\n--- Kích thước Từ điển ---")
print(f"Kích thước từ điển từ (VOCAB_SIZE): {VOCAB_SIZE}")
print(f"Kích thước từ điển nhãn (OUTPUT_SIZE): {OUTPUT_SIZE}")

Đang tải bộ dữ liệu CoNLL 2003 (phiên bản ổn định)...
Tải dữ liệu hoàn tất.
DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

Các nhãn NER (9 classes): ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

--- Kích thước Từ điển ---
Kích thước từ điển từ (VOCAB_SIZE): 23625
Kích thước từ điển nhãn (OUTPUT_SIZE): 9


In [6]:
# Cell 3: Task 2 - Tạo PyTorch Dataset và DataLoader
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np

# Giá trị đặc biệt để đệm nhãn. Phải là giá trị không được dùng trong tag_to_ix
PADDING_TAG_INDEX = -1

# 1. Tạo lớp NERDataset
class NERDataset(Dataset):
    def __init__(self, sentences, tags, word_to_ix, tag_to_ix):
        self.sentences = sentences
        self.tags = tags
        self.word_to_ix = word_to_ix
        self.tag_to_ix = tag_to_ix

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        tags = self.tags[idx]

        # Chuyển đổi từ/nhãn sang chỉ số, dùng <UNK> cho từ không biết
        sentence_indices = [self.word_to_ix.get(word, self.word_to_ix["<UNK>"]) for word in sentence]
        # Chuyển đổi nhãn sang chỉ số
        tag_indices = [self.tag_to_ix[tag] for tag in tags]

        return (torch.tensor(sentence_indices, dtype=torch.long),
                torch.tensor(tag_indices, dtype=torch.long))

# 2. Tạo DataLoader và Hàm collate_fn
def collate_fn(batch):
    sentences = [item[0] for item in batch]
    tags = [item[1] for item in batch]

    # Đệm (padding) các câu. Sử dụng index của <PAD> (0)
    sentences_padded = pad_sequence(sentences, batch_first=True, padding_value=word_to_ix["<PAD>"])

    # Đệm (padding) các nhãn. Sử dụng giá trị đặc biệt -1
    tags_padded = pad_sequence(tags, batch_first=True, padding_value=PADDING_TAG_INDEX)

    return sentences_padded, tags_padded

# Khởi tạo Datasets
# Biến train_sentences, train_tags, val_sentences, val_tags được khởi tạo ở Cell 2
train_dataset = NERDataset(train_sentences, train_tags, word_to_ix, tag_to_ix)
val_dataset = NERDataset(val_sentences, val_tags, word_to_ix, tag_to_ix)

# Khởi tạo DataLoaders
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

print(f"Số lượng mẫu huấn luyện: {len(train_dataset)}")
print(f"Số lượng batch trong Train DataLoader: {len(train_loader)}")

Số lượng mẫu huấn luyện: 14041
Số lượng batch trong Train DataLoader: 439


In [7]:
# Cell 4: Task 3 - Xây dựng Mô hình RNN (Bi-LSTM)
import torch.nn as nn
import torch.optim as optim

# Định nghĩa kiến trúc Bi-LSTM
class SimpleRNNForTokenClassification(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size):
        super(SimpleRNNForTokenClassification, self).__init__()

        # 1. Lớp Embedding
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # 2. Lớp LSTM (Bi-directional)
        # Bidirectional=True sẽ tạo ra output có kích thước 2 * hidden_dim
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)

        # 3. Lớp Linear để ánh xạ từ hidden state sang không gian nhãn
        self.linear = nn.Linear(hidden_dim * 2, output_size)

    def forward(self, sentences):
        embedded = self.embedding(sentences)
        # rnn_output: (batch_size, seq_len, 2 * hidden_dim)
        rnn_output, _ = self.rnn(embedded)
        # scores: (batch_size, seq_len, output_size)
        scores = self.linear(rnn_output)

        return scores

# Khởi tạo tham số mô hình
EMBEDDING_DIM = 100
HIDDEN_DIM = 256

# Khởi tạo mô hình và chuyển sang GPU/CPU
# Các biến VOCAB_SIZE, OUTPUT_SIZE được khởi tạo ở Cell 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleRNNForTokenClassification(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_SIZE).to(device)

print(model)
print(f"Mô hình được chuyển sang thiết bị: {device}")

SimpleRNNForTokenClassification(
  (embedding): Embedding(23625, 100)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=9, bias=True)
)
Mô hình được chuyển sang thiết bị: cpu


In [8]:
# Cell 5: Task 4 - Huấn luyện Mô hình
LEARNING_RATE = 0.005
NUM_EPOCHS = 5

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Loss function: nn.CrossEntropyLoss
# ignore_index=PADDING_TAG_INDEX (-1): bỏ qua loss của các token đệm
# PADDING_TAG_INDEX được định nghĩa ở Cell 3
loss_function = nn.CrossEntropyLoss(ignore_index=PADDING_TAG_INDEX)

print("\n--- Bắt đầu Huấn luyện Mô hình ---")

for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    total_loss = 0

    for sentences, tags in train_loader:
        sentences = sentences.to(device)
        tags = tags.to(device)

        # (1) Xóa gradient cũ
        optimizer.zero_grad()

        # (2) Forward pass
        scores = model(sentences)

        # Reshape cho CrossEntropyLoss: (Batch * Seq_len, Output_size) và (Batch * Seq_len)
        scores = scores.view(-1, scores.shape[-1])
        tags = tags.view(-1)

        # (3) Tính loss
        loss = loss_function(scores, tags)
        total_loss += loss.item()

        # (4) Backward pass
        loss.backward()

        # (5) Cập nhật trọng số
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch}/{NUM_EPOCHS}, Loss trung bình: {avg_loss:.4f}")

print("--- Huấn luyện hoàn tất ---")


--- Bắt đầu Huấn luyện Mô hình ---
Epoch 1/5, Loss trung bình: 0.3331
Epoch 2/5, Loss trung bình: 0.0788
Epoch 3/5, Loss trung bình: 0.0182
Epoch 4/5, Loss trung bình: 0.0052
Epoch 5/5, Loss trung bình: 0.0017
--- Huấn luyện hoàn tất ---


In [9]:
# Cell 6: Task 5 - Đánh giá Mô hình và Dự đoán
from seqeval.metrics import classification_report

# 1. Viết hàm evaluate
def evaluate(model, data_loader, device):
    model.eval()
    total_correct = 0
    total_non_padding_tokens = 0
    all_predictions = []
    all_true_labels = []

    with torch.no_grad():
        for sentences, tags in data_loader:
            sentences = sentences.to(device)
            tags = tags.to(device)

            # Forward pass
            scores = model(sentences)
            # Lấy dự đoán
            predictions = torch.argmax(scores, dim=-1)

            # Lọc các vị trí padding
            # PADDING_TAG_INDEX được định nghĩa ở Cell 3
            mask = (tags != PADDING_TAG_INDEX)

            # Tính toán độ chính xác (Token Accuracy)
            correct = (predictions == tags) & mask
            total_correct += correct.sum().item()
            total_non_padding_tokens += mask.sum().item()

            # Chuẩn bị cho seqeval
            for i in range(sentences.shape[0]):
                # Lấy độ dài thực của câu (không tính padding)
                sentence_len = (tags[i] != PADDING_TAG_INDEX).sum().item()

                true_sentence_tags = tags[i, :sentence_len].cpu().numpy()
                pred_sentence_tags = predictions[i, :sentence_len].cpu().numpy()

                # Chuyển index thành tên nhãn string (sử dụng ix_to_tag từ Cell 2)
                all_true_labels.append([ix_to_tag[t] for t in true_sentence_tags])
                all_predictions.append([ix_to_tag[p] for p in pred_sentence_tags])

    # 2. Báo cáo kết quả
    token_accuracy = total_correct / total_non_padding_tokens if total_non_padding_tokens > 0 else 0
    print(f"\nĐộ chính xác trên tập validation (Token Accuracy): {token_accuracy:.4f}")

    print("\n--- Báo cáo NER chi tiết (seqeval) ---")
    # seqeval tính Precision, Recall, F1-score cho từng loại thực thể
    print(classification_report(all_true_labels, all_predictions, zero_division=0))

    return token_accuracy

# Đánh giá mô hình trên tập validation
val_accuracy = evaluate(model, val_loader, device)


# 3. Viết hàm predict_sentence
def predict_sentence(sentence_str, model, word_to_ix, ix_to_tag, device):
    model.eval()

    tokens = sentence_str.split()
    # Chuyển đổi từ thành index, dùng <UNK> nếu từ không có trong từ điển (sử dụng word_to_ix từ Cell 2)
    token_indices = [word_to_ix.get(word, word_to_ix["<UNK>"]) for word in tokens]

    input_tensor = torch.tensor([token_indices], dtype=torch.long).to(device)

    with torch.no_grad():
        scores = model(input_tensor)
        # Lấy nhãn có điểm số cao nhất
        predictions = torch.argmax(scores, dim=-1).squeeze(0).cpu().numpy()

    predicted_tags = [ix_to_tag[p] for p in predictions]

    # In ra các cặp (từ, nhãn dự đoán)
    print("\n--- Ví dụ dự đoán câu mới ---")
    print(f"Câu: {sentence_str}")
    print("Dự đoán:")
    for word, tag in zip(tokens, predicted_tags):
        print(f"({word}, {tag})")

# Ví dụ dự đoán câu mới (các thực thể CoNLL 2003: PER, ORG, LOC, MISC)
new_sentence = "The New York Times reported that Joe Biden visited London yesterday"
predict_sentence(new_sentence, model, word_to_ix, ix_to_tag, device)


Độ chính xác trên tập validation (Token Accuracy): 0.9547

--- Báo cáo NER chi tiết (seqeval) ---
              precision    recall  f1-score   support

         LOC       0.93      0.81      0.86      1837
        MISC       0.89      0.77      0.82       922
         ORG       0.51      0.81      0.63      1341
         PER       0.77      0.70      0.74      1842

   micro avg       0.74      0.77      0.75      5942
   macro avg       0.77      0.77      0.76      5942
weighted avg       0.78      0.77      0.76      5942


--- Ví dụ dự đoán câu mới ---
Câu: The New York Times reported that Joe Biden visited London yesterday
Dự đoán:
(The, B-ORG)
(New, I-ORG)
(York, I-ORG)
(Times, I-ORG)
(reported, O)
(that, O)
(Joe, B-PER)
(Biden, O)
(visited, O)
(London, B-LOC)
(yesterday, O)
